In [2]:
from pathlib import Path

from document_ai.utils import pdf2image

REPO_DIR = Path().absolute().parent

filepath = REPO_DIR / "data/documents/deepseek-ocr-2.pdf"
output_dir = REPO_DIR / "data/output"

In [8]:
images = pdf2image.convert_pdf_to_images(filepath)
image_path = REPO_DIR / "data/images/page.png"
images[8].save(image_path)

In [4]:
from transformers import AutoModel, AutoTokenizer
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
model_name = 'deepseek-ai/DeepSeek-OCR-2'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True, use_safetensors=True)
model = model.eval().cuda().to(torch.bfloat16)

/workspace/document-ai/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR2. This is not supported for all configurations of models and can yield errors.


In [11]:
# prompt = "<image>\nFree OCR. "
prompt = "<image>\n<|grounding|>Convert tables to markdown. "
image_file = str(image_path)

res = model.infer(
    tokenizer,
    prompt=prompt,
    image_file=image_file,
    output_path=output_dir,
    base_size=1024,
    image_size=768,
    crop_mode=True,
    save_results=True,
)


/workspace/document-ai/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([6, 144, 1280])
<|ref|>figure_title<|/ref|><|det|>[[112, 99, 883, 165]]<|/det|>
Table 1 | Comprehensive evaluation of document reading on OmniDocBench v1.5. V-token \( ^{max} \)  represents the maximum number of visual tokens used per page in this benchmark. R-order denotes reading order. Except for DeepSeek OCR and DeepSeek OCR 2, all other model results in this table are sourced from the OmniDocBench repository.

<|ref|>table<|/ref|><|det|>[[119, 175, 872, 594]]<|/det|>
<table><tr><td>Model</td><td>\( |V-\text{token}^{max}\downarrow|Overall\uparrow|Text^{Edit}\downarrow Formula^{CDM}\uparrow Table^{TEDs}\uparrow Table^{TEDs_{s}}\uparrow R-order^{Edit}\uparrow \)</td></tr><tr><td colspan="3">Pipeline</td></tr><tr><td>Marker-1.8.2 [1]</td><td>-</td></tr><tr><td>MinerU2-pp [45]</td><td>-</td></tr><tr><td>Dolphin [17]</td><td>-</td></tr><tr><td>Dolphin-1.5 [17]</td><td>-</td></tr><tr><td>PP-StructureV3 [13]</td><td>-</td></tr><tr><td

image: 0it [00:00, ?it/s]
other: 100%|██████████| 6/6 [00:00<00:00, 84449.07it/s]
